In [12]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import tensorflow as tf

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def unpack_im(im_vector):
    im = np.zeros([32,32,3])
    for i in range(3):
        im[:,:,i] = im_vector[i*1024:i*1024+1024].reshape([32,32])
    im = im.astype('uint8')
    return im

def val2bin(labels):
    bin = np.zeros([len(labels),10])
    for i in range(len(labels)):
        bin[i, labels[i]]=1
    return bin.astype('uint')

dict = unpickle('./test_batch')
for key in dict:
    print(key)
print(dict[b'batch_label'])
labels = np.array(dict[b'labels'])
#labels = val2bin(labels)
data = np.array(dict[b'data'])
print(labels.shape, data.shape)

#demonstrate images
for i in range(2):
    im = unpack_im(data[i,:])
    plt.imshow(im), plt.show()
    
# divide to train and test
train_percent = 0.7
train_num = int(len(data)*train_percent)
x_train = data[:train_num]
y_train = labels[:train_num]
x_test = data[train_num:]
y_test = labels[train_num:]

    
#train the model
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
#   tf.keras.layers.Conv2D(32, kernel_size=(8, 8), strides=(1, 1),
#                  activation=tf.nn.relu), ÷
    
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


print('train',x_train.shape, y_train.shape)
print('test',x_test.shape, y_test.shape)
model.fit(x_train, y_train, epochs=10)
model.evaluate(x_test, y_test)


b'batch_label'
b'labels'
b'data'
b'filenames'
b'testing batch 1 of 1'
(10000,) (10000, 3072)


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

train (7000, 3072) (7000,)
test (3000, 3072) (3000,)
Epoch 1/10


KeyboardInterrupt: 

In [7]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np

# extract the features first and then classify

base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

img_path = 'round1/1011_ship.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)
print(block4_pool_features.shape)

(1, 14, 14, 512)


In [11]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)

img_path = 'round1/1011_ship.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
print(features.shape)

(1, 7, 7, 512)
